In [49]:
# En esta celda se puede probar la funcion de analisis de una imagen

from lib.image import image_data
from lib.analysis import dyssynchrony_analysis
import cv2

# Path completo donde esta la imagen (incluyendo nombre y extencion)
path = '/home/leandro/Documentos/Analisis_de_imagenes/CardiAP/DesktopApp/photos_examples/'
photo_name = 'c3e000'
photo = photo_name + '.tif'
# Ancho de la feta a analisar (es un parametro opcional)
slice_width = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 70

# Calibracion del tiempo de cada pixel
calibration = 3.1

image = cv2.imread(path + photo_name + '.tif')
# Select ROI
fromCenter = False
showCrosshair = False
seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

image = image_data.get_image_data(image)
image = image_data.crop_vertical(image, x_start, x_end)
image = image_data.crop_horizontal(image, y_start, y_end)

results = dyssynchrony_analysis.analyze_image(image, min_dist_between_maxs, calibration, slice_width=slice_width)

/home/leandro/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


# PRUEBA ANALISIS 

In [ ]:
# En esta celda se puede probar la funcion de analisis todas las imagenes .tif en un directorio
import os
from lib.image import image_data
from lib.analysis import dyssynchrony_analysis
import cv2

# Ancho de la feta a analisar (es un parametro opcional)
slice_width = 5

# Distancia minima en pixeles entre picos
min_dist_between_maxs = 200

# Calibracion del tiempo de cada pixel
calibration = 3.1

# Path donde estan las imagenes
path = "C:/Users/Leand/OneDrive/Documentos/Lean/Analizador_imagenes_calcio/1 Hz/"

images_paths = [ f'{path}/{file}' for file in os.listdir(path) if file.endswith(".tif") ]
images = [ cv2.imread(image_path) for image_path in images_paths ]

In [ ]:
fromCenter = False
showCrosshair = False

#Tomamos la primer imagen para seleccionar el recorte elegido el resto de las imagenes se van a recortar igual
seleted_parameters = cv2.selectROI(images[0], fromCenter, showCrosshair)

# Crop image
x_start = int(seleted_parameters[1])
x_end = x_start + int(seleted_parameters[3])
y_start = int(seleted_parameters[0])
y_end = y_start + int(seleted_parameters[2])

cv2.destroyAllWindows()

In [ ]:
images = [ image_data.get_image_data(image) for image in images ]
images = [ image_data.crop_vertical(image, x_start, x_end) for image in images ]
images = [ image_data.crop_horizontal(image, y_start, y_end) for image in images ]

In [ ]:
results = [(dyssynchrony_analysis.analyze_image(image, min_dist_between_maxs, calibration, slice_width=5), print ('Done')) for image in images]

# Analisis de datos
Notas: 
- Image corresponde al análisis de toda la célula y slices corresponde a fetas de esa imagen
- intensidades es el valor que resulta de comprimir (sumando) la matriz con las coordenadas de x,y,z de los pixeles 
- max_peaks_pos es la index de en la lista de intensidades 
- max_peaks_intensities el valor de intensidad que se corresponde con el pico en la posicion analoga de max_peaks_pos

In [50]:
#Imports
import json
import numpy as np
import pandas as pd

In [51]:
#encodes a dictionary into a jason
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)


def jasonGenerator(path,results,photo_name):
    """ Generates a .jason file in the path given folder from the a dictionary """
    with open(path + '/analysis_result_' + photo_name + '.json',"w") as miarch:
        miarch.write(json.dumps(results, cls=NumpyEncoder))

In [52]:
def jasonReader(path, photo):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open( path + 'analysis_result_' + photo+ '.json','r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [53]:
def slicesParser_max_peaksI(path):
#     read the analysis result 
    dictres = jasonReader(path,photo_name)

#extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x) for x in range(0,len(allSlices[0]['max_peaks_intensities']))]    
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        for i in range(0,len(allSlices)):
            df_sum.loc[i] = allSlices[i]['max_peaks_intensities'][0:] 
        return df_sum

    except:
        print('no possible callculation')

In [54]:
def slicesParser_halfpeakstime(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']
#     try:
    #seting columns names
    column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_half_peaks']))]
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in range(0,len(allSlices)):
        df_sum.loc[i+1] = allSlices[i]['times_to_half_peaks']
    return df_sum

#     except:
#         print('no possible callculation')

In [55]:
def slicesParser_amplitudes(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['amplitudes']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['amplitudes']

        return df_sum
    
    except:
        print('no possible callculation')

In [56]:
def slicesParser_min_peaksI(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']
    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['min_peaks_intensities']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['min_peaks_intensities']

        return df_sum
    
    except:
        print('no possible callculation')

In [57]:
def slicesParser_peaktime(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']

    try:
        #seting columns names
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['times_to_peaks']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['times_to_peaks']

        return df_sum
    
    except:
        print('no possible callculation')

In [58]:
def slicesParser_tau(path):
    #read the analysis result 
    dictres = jasonReader(path,photo_name)
    #extract all the slices data
    allSlices = dictres['slices']
    #seting columns names
    try:
        column_names = ['transient' + str(x+1) for x in range(0,len(allSlices[0]['tau_s']))]
        #define a dataframe
        df_sum = pd.DataFrame(columns=column_names)
        #populating the dataframe
        for i in range(0,len(allSlices)):
            df_sum.loc[i+1] = allSlices[i]['tau_s']
        
        return df_sum
    
    except:
        print('no possible callculation')

In [59]:
def BasicStasts(df):
    if df.empty == False:
        return df.astype('int').describe()

In [60]:
def discordIndex(df):
    return df.iloc[2].mean()

In [61]:
def amplitudes_ratio(df):
    df_alt = pd.DataFrame()
    for i in range(0,len(list(df))-1):
        name = 'ratio' + str(i+1) +'-' + str(i)
        df_alt[name] = df[list(df)[i+1]]/df[list(df)[i]]
    return df_alt

In [62]:
jasonGenerator(path,results,photo_name)

In [77]:
path = '/home/leandro/Documentos/Analisis_de_imagenes/CardiAP/DesktopApp/photos_examples/'
photo_name = 'c1d000'
photo = photo_name + '.tif'

In [78]:
loaded_dict = jasonReader(path,photo_name)

In [79]:
BasicStasts(slicesParser_max_peaksI(path))

,transient0,transient1,transient2,transient3,transient4,transient5
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,47.510638,47.127660,47.382979,46.787234,46.936170,46.893617
std,3.236157,3.486727,2.658483,3.348823,3.454073,2.868374
min,41.000000,40.000000,42.000000,41.000000,39.000000,39.000000
25%,46.000000,45.500000,46.000000,45.000000,45.000000,45.000000
50%,47.000000,46.000000,47.000000,46.000000,47.000000,47.000000
75%,50.000000,49.000000,48.000000,50.000000,49.000000,50.000000
max,54.000000,54.000000,53.000000,54.000000,55.000000,52.000000


In [80]:
amplitudes_ratio(slicesParser_amplitudes(path)).std()

ratio1-0    0.170070
ratio2-1    0.156315
ratio3-2    0.170354
ratio4-3    0.187402
ratio5-4    0.173035
dtype: float64

In [81]:
amp_mean = slicesParser_amplitudes(path).mean()
for i in range(0,len(amp_mean)-1):
    AR_ind = (1- (amp_mean[i+1])/amp_mean[i])
    print (AR_ind)

-0.009899345564696294
-0.05050426651560547
0.13510295336402955
-0.07285565740647026
0.009006912821047597


In [82]:
slicesParser_min_peaksI(path).head(2)

,transient1,transient2,transient3,transient4,transient5,transient6,transient7
1,19,19,21,20,18,20,18
2,20,19,20,21,19,21,17


In [83]:
slicesParser_tau(path).head()

,transient1,transient2,transient3,transient4,transient5,transient6
1,4.760239,6.089710,7.471964,9.411866,11.251575,12.253049
2,4.771013,6.252272,7.352561,9.033019,11.031760,12.250300
3,4.761461,6.320503,7.316683,9.065048,10.353530,12.398747
4,4.743965,6.380447,7.395527,9.103748,10.180424,12.475590
5,4.729876,6.355306,7.549130,9.248208,10.124134,12.247318


In [84]:
DI = pd.DataFrame()
for j in range (1, len(slicesParser_amplitudes(path).columns)):
    rel_diff = []
    for i in range (0, len (slicesParser_amplitudes(path))):
        T1 = list(slicesParser_amplitudes(path)['transient'+str(j)])[i]
        T2 = list(slicesParser_amplitudes(path)['transient'+str(j+1)])[i]
        rel_diff.append((T1-T2)/max(T1,T2))
    DI[j] = rel_diff

In [99]:
AR = []
for j in range (1, len(slicesParser_amplitudes(path).columns)):
    T1_mean = float(slicesParser_amplitudes(path)['transient'+str(j)].mean())
    T2_mean = float(slicesParser_amplitudes(path)['transient'+str(j+1)].mean())
    AR.append((T1_mean-T2_mean)/T1_mean)

In [85]:
DI.std().mean()

0.15144198950783894

In [103]:
AR_mean = sum(AR)/len(AR) 

In [104]:
AR_mean

0.0021701193396610518